In [1]:
import pandas as pd
import glob
import re
import numpy as np

In [2]:
bonds = glob.glob('bonds\*.csv')

In [3]:
bonds

['bonds\\Eurex Swiss CHF Government Bond Future Historical Data.csv',
 'bonds\\Euribor Futures Historical Data.csv',
 'bonds\\Euro BOBL Futures Historical Data.csv',
 'bonds\\Euro BTP Futures Historical Data.csv',
 'bonds\\Euro Bund Futures Historical Data.csv',
 'bonds\\Euro OAT Futures Historical Data.csv',
 'bonds\\Euro SCHATZ Futures Historical Data.csv',
 'bonds\\Euro-Buxl Futures Historical Data.csv',
 'bonds\\Japan Government Bond Futures Historical Data.csv',
 'bonds\\Short Sterling Futures Historical Data.csv',
 'bonds\\UK Gilt Futures Historical Data.csv',
 'bonds\\US 10 Year T-Note Futures Historical Data.csv',
 'bonds\\US 2 Year T-Note Futures Historical Data.csv',
 'bonds\\US 30 Year T-Bond Futures Historical Data.csv',
 'bonds\\US 5 Year T-Note Futures Historical Data.csv']

In [4]:
l = []
for i in range(len(bonds)):
    bond_name = '_'.join(bonds[i].replace('\\', ' ').split()[1:3])
    temp = pd.read_csv(bonds[i], names=['Date', bond_name], usecols=['Date', bond_name], 
                     header = 0, parse_dates=['Date'], index_col='Date')
    l.append(temp)
    
df = pd.concat(l, axis=1)

In [5]:
df

,Eurex_Swiss,Euribor_Futures,Euro_BOBL,Euro_BTP,Euro_Bund,Euro_OAT,Euro_SCHATZ,Euro-Buxl_Futures,Japan_Government,Short_Sterling,UK_Gilt,US_10,US_2,US_30,US_5
Date,,,,,,,,,,,,,,,
2014-01-01,NaN,99.72,124.46,114.78,139.24,NaN,110.295,NaN,143.13,99.43,106.56,NaN,109.91,128.01,119.25
2014-01-02,NaN,99.74,124.41,115.81,139.12,131.55,110.295,121.60,143.26,99.43,106.52,123.22,109.91,128.62,119.41
2014-01-03,143.66,99.74,124.45,116.05,139.09,131.62,110.295,121.40,143.27,99.43,106.60,123.17,109.88,128.53,119.35
2014-01-06,144.31,99.72,124.61,116.13,139.54,131.88,110.295,121.96,143.54,99.44,107.21,123.47,109.90,129.03,119.51
2014-01-07,144.57,99.71,124.72,116.68,139.76,132.22,110.295,122.28,143.81,NaN,107.44,123.69,109.91,129.31,119.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-25,172.24,100.53,135.44,151.32,175.33,169.91,112.345,226.22,152.14,99.95,135.08,138.27,110.40,172.88,125.57
2020-11-26,172.49,100.53,135.48,151.40,175.51,170.16,112.355,227.00,152.10,99.96,135.44,138.45,110.41,173.45,125.61
2020-11-27,172.60,100.53,135.49,151.50,175.50,170.13,112.350,227.16,152.05,99.96,135.26,138.55,110.41,173.88,125.67


In [6]:
df.isna().sum()

Eurex_Swiss          691
Euribor_Futures      107
Euro_BOBL            124
Euro_BTP             125
Euro_Bund            125
Euro_OAT             129
Euro_SCHATZ          125
Euro-Buxl_Futures    108
Japan_Government      66
Short_Sterling       196
UK_Gilt              137
US_10                 70
US_2                  61
US_30                 65
US_5                  64
dtype: int64

In [7]:
df.drop('Eurex_Swiss', axis=1, inplace=True)
df = df.ffill().bfill()
df.isna().sum()

Euribor_Futures      0
Euro_BOBL            0
Euro_BTP             0
Euro_Bund            0
Euro_OAT             0
Euro_SCHATZ          0
Euro-Buxl_Futures    0
Japan_Government     0
Short_Sterling       0
UK_Gilt              0
US_10                0
US_2                 0
US_30                0
US_5                 0
dtype: int64

In [8]:
df

,Euribor_Futures,Euro_BOBL,Euro_BTP,Euro_Bund,Euro_OAT,Euro_SCHATZ,Euro-Buxl_Futures,Japan_Government,Short_Sterling,UK_Gilt,US_10,US_2,US_30,US_5
Date,,,,,,,,,,,,,,
2014-01-01,99.72,124.46,114.78,139.24,131.55,110.295,121.60,143.13,99.43,106.56,123.22,109.91,128.01,119.25
2014-01-02,99.74,124.41,115.81,139.12,131.55,110.295,121.60,143.26,99.43,106.52,123.22,109.91,128.62,119.41
2014-01-03,99.74,124.45,116.05,139.09,131.62,110.295,121.40,143.27,99.43,106.60,123.17,109.88,128.53,119.35
2014-01-06,99.72,124.61,116.13,139.54,131.88,110.295,121.96,143.54,99.44,107.21,123.47,109.90,129.03,119.51
2014-01-07,99.71,124.72,116.68,139.76,132.22,110.295,122.28,143.81,99.44,107.44,123.69,109.91,129.31,119.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-25,100.53,135.44,151.32,175.33,169.91,112.345,226.22,152.14,99.95,135.08,138.27,110.40,172.88,125.57
2020-11-26,100.53,135.48,151.40,175.51,170.16,112.355,227.00,152.10,99.96,135.44,138.45,110.41,173.45,125.61
2020-11-27,100.53,135.49,151.50,175.50,170.13,112.350,227.16,152.05,99.96,135.26,138.55,110.41,173.88,125.67


In [9]:
df.to_csv('bonds.csv')